# Imports

In [4]:
import firebase_admin
from firebase_admin import credentials, firestore
import smtplib
import ssl
from email.message import EmailMessage
import os

# Define sender and receiver

In [5]:
# Initialize Firebase Admin SDK
cred = credentials.Certificate('./firebase.json')
firebase_admin.initialize_app(cred)

# Initialize Firestore database
db = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

# Function to send email

In [10]:
# Function to send email
def send_email(email_receiver):
    email_sender = 'noreply.watermeloncup@gmail.com'
    email_password = os.environ.get('EMAIL_PASSWORD')
    subject = '[Action Required] Register for the Watermelon Cup!'
    body = """
    Hello - you have not registered for the watermelon cup yet. 
    Click on the link below to register for the Watermelon Cup:\n
    \n
    home page > login > click on register at the bottom of the homepage > fill out the form > register\n
    \n
    https://www.watermeloncup.com\n
    \n
    Thank you!
    """
    
    em = EmailMessage()
    em['From'] = email_sender
    em['To'] = email_receiver
    em['Subject'] = subject
    em.set_content(body)

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
        smtp.login(email_sender, email_password)
        smtp.sendmail(email_sender, email_receiver, em.as_string())

# Get users who haven't registered yet

In [11]:
# Get users from Firestore where registered=False
users_ref = db.collection('users')
query = users_ref.where('registered', '==', False)
unregistered_users = query.stream()

# Print list of unregistered email addresses
unregistered_emails = [user.to_dict()['email'] for user in unregistered_users]
print("List of unregistered email addresses:")
for email in unregistered_emails:
    print(email)

List of unregistered email addresses:
greenshayna@gmail.com
muellerboys83@gmail.com
ldamore@westportps.org
dylanhoke122@gmail.com
wmcd9999@gmail.com
muri.moreno@hotmail.com
jpcorridon@gmail.com
lucas.alarcon.frias@gmail.com
dutchschlegelmilch@gmail.com


C:\Users\mdj20\AppData\Roaming\Python\Python312\site-packages\google\cloud\firestore_v1\base_collection.py:302: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


# Main - send emails to each unregistered users

In [12]:
# Send email to each unregistered user
for email in unregistered_emails:
    send_email(email)
    print(f"Email sent to {email}")

SMTPResponseException: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials d75a77b69052e-43e19256258sm133302721cf.49 - gsmtp')